In [13]:
"""
Full training script for NSLT-100 using ST-GCN + Temporal Self-Attention.
- Expects:
  - LANDMARK_FILE: .npz of landmarks (keys are video ids)
  - SPLIT_FILE: nslt_100.json (same format as your nslt_300.json)
- Produces:
  - best_stgcn_100.pth
  - training_log.csv
  - training_curves.png

Notes / fixes compared to your earlier script:
- Builds a label->index mapping from the train split so labels become contiguous 0..(num_classes-1)
- Uses CosineAnnealingLR correctly (scheduler.step() each epoch)
- Computes class weights from mapped labels
- Computes top-1 and top-5 accuracy
- Safer handling of landmark array shapes
- Slightly reduced num_workers default for compat

Run on Kaggle / local with GPU. Adjust paths and hyperparams at the top.
"""

import os
import json
import math
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.utils.class_weight import compute_class_weight

# --------------------------
# Config
# --------------------------
LANDMARK_FILE = '/kaggle/input/new-bones-joints/Landmarks_GCN_augmented.npz'
SPLIT_FILE = '/kaggle/input/json-files/nslt_100.json'
NUM_CLASSES = 100
TOTAL_LANDMARKS = 67  # 15 pose + 2 mid + 42 hands + 8 face = 67
C = 4                # x,y,z + (optionally) confidence or reserved channel
T_FRAMES = 128
BATCH_SIZE = 32
EPOCHS = 200
BASE_LR = 3e-4
WEIGHT_DECAY = 1e-4
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
USE_AMP = True
MIXUP_ALPHA = 0.4
LABEL_SMOOTHING = 0.1
NUM_WORKERS = 2

# reproducibility (not strictly deterministic)
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# --------------------------
# Helpers (temporal resample + preprocessor)
# --------------------------

def temporal_resample(sequence, target_len):
    orig_len = sequence.shape[0]
    if orig_len == target_len:
        return sequence
    if orig_len == 1:
        return np.repeat(sequence, target_len, axis=0)
    target_positions = np.linspace(0, orig_len - 1, target_len)
    old_idx = np.arange(orig_len)
    out = np.zeros((target_len, sequence.shape[1], sequence.shape[2]), dtype=sequence.dtype)
    for v in range(sequence.shape[1]):
        for c in range(sequence.shape[2]):
            values = sequence[:, v, c]
            out[:, v, c] = np.interp(target_positions, old_idx, values)
    return out


class LandmarkPreprocessor:
    def __init__(self, augment=True, target_frames=T_FRAMES):
        self.augment = augment
        self.target_frames = target_frames
        # indices
        self.POSE_IDXS = list(range(0, 17))  # includes 2 mid points assumption
        self.LEFT_HAND_IDXS = list(range(17, 38))
        self.RIGHT_HAND_IDXS = list(range(38, 59))
        self.FACE_IDXS = list(range(59, 67))

    def normalize_frame(self, frame):
        normalized = frame.copy()
        # Pose + face
        try:
            pose = frame[self.POSE_IDXS, :2]
            if np.sum(np.abs(pose)) > 0:
                nose = pose[0]
                l_s = pose[1] if len(pose) > 1 else nose
                r_s = pose[2] if len(pose) > 2 else nose + 0.1
                if np.all(l_s == 0) or np.all(r_s == 0):
                    l_s = np.nan_to_num(np.mean(pose, axis=0))
                    r_s = l_s + 0.1
                dist = np.linalg.norm(l_s - r_s)
                if dist < 1e-4:
                    dist = 0.1
                hu = dist / 2.0
                bw = 6 * hu
                bh = 7 * hu
                bx = nose[0] - 3 * hu
                by = nose[1] - 0.5 * hu
                pf_idxs = self.POSE_IDXS + self.FACE_IDXS
                normalized[pf_idxs, 0] = (frame[pf_idxs, 0] - bx) / max(bw, 1e-6) - 0.5
                normalized[pf_idxs, 1] = (frame[pf_idxs, 1] - by) / max(bh, 1e-6) - 0.5
        except Exception:
            pass

        # hands
        for h_idxs in [self.LEFT_HAND_IDXS, self.RIGHT_HAND_IDXS]:
            try:
                hand = frame[h_idxs]
                if np.sum(np.abs(hand)) > 0:
                    xs, ys = hand[:, 0], hand[:, 1]
                    if (np.max(xs) != np.min(xs)) or (np.max(ys) != np.min(ys)):
                        xmin, xmax, ymin, ymax = np.min(xs), np.max(xs), np.min(ys), np.max(ys)
                        box_size = max(xmax - xmin, ymax - ymin, 1e-4)
                        normalized[h_idxs, 0] = (hand[:, 0] - xmin) / box_size - 0.5
                        normalized[h_idxs, 1] = (hand[:, 1] - ymin) / box_size - 0.5
            except Exception:
                continue

        # z normalize by torso median
        try:
            torso_z = frame[self.POSE_IDXS, 2]
            torso_z_nonzero = torso_z[torso_z != 0]
            if len(torso_z_nonzero) > 0:
                median_z = np.median(torso_z_nonzero)
                normalized[:, 2] = (frame[:, 2] - median_z)
        except Exception:
            pass

        return normalized

    def augment_sequence(self, seq):
        # rotation noise
        angle = random.uniform(-12, 12)
        rad = math.radians(angle)
        cos, sin = math.cos(rad), math.sin(rad)
        R = np.array([[cos, -sin], [sin, cos]], dtype=np.float32)
        xy = seq[:, :, :2].reshape(-1, 2)
        xy = xy @ R.T
        seq[:, :, :2] = xy.reshape(seq.shape[0], seq.shape[1], 2)
        seq += np.random.normal(0, 0.003, size=seq.shape).astype(np.float32)

        # temporal jitter
        if random.random() < 0.5:
            T = seq.shape[0]
            jittered = []
            for t in range(T):
                if random.random() < 0.05 and t < T - 1:
                    continue
                jittered.append(seq[t])
                if random.random() < 0.05:
                    jittered.append(seq[t])
            if len(jittered) < 2:
                jittered = [seq[t] for t in range(T)]
            jittered = np.stack(jittered, axis=0)
            seq = temporal_resample(jittered, self.target_frames)
        return seq

    def __call__(self, landmark_sequence: np.ndarray, is_train: bool):
        landmark_sequence = np.nan_to_num(landmark_sequence, 0.0).astype(np.float32)
        seq = temporal_resample(landmark_sequence, self.target_frames)
        seq = np.stack([self.normalize_frame(frame) for frame in seq], axis=0)
        if is_train and self.augment:
            seq = self.augment_sequence(seq)
        return seq


# --------------------------
# Dataset
# --------------------------
class WLASLLandmarkDataset(Dataset):
    def __init__(self, landmark_path, split_file_path, split='train', preprocessor=None, label_map=None):
        self.split = split
        self.preprocessor = preprocessor if preprocessor is not None else LandmarkPreprocessor()
        landmarks_data = dict(np.load(landmark_path, allow_pickle=True))
        with open(split_file_path, 'r') as f:
            split_data = json.load(f)
        split_dict = split_data.get('root', split_data)
        self.samples = []

        for video_id, info in split_dict.items():
            subset = info.get('subset') or info.get('split') or info.get('subset', None)
            if subset == self.split:
                if video_id in landmarks_data:
                    action = info.get('action', [])
                    if not isinstance(action, list) or len(action) == 0:
                        continue
                    raw_label = action[0]
                    # map to contiguous label if mapping provided
                    if label_map is None:
                        label = int(raw_label)
                    else:
                        if str(raw_label) not in label_map:
                            continue
                        label = int(label_map[str(raw_label)])
                    landmarks = landmarks_data[video_id]
                    self.samples.append({"video_id": video_id, "landmarks": landmarks, "label": label})
        print(f"✅ Created '{self.split}' split with {len(self.samples)} samples.")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        landmark_sequence = sample["landmarks"]
        # handle flattened format
        if isinstance(landmark_sequence, np.ndarray) and landmark_sequence.ndim == 2 and landmark_sequence.shape[1] == TOTAL_LANDMARKS * 3:
            landmark_sequence = landmark_sequence.reshape(-1, TOTAL_LANDMARKS, 3)
        processed = self.preprocessor(landmark_sequence, is_train=(self.split == 'train'))
        processed = np.transpose(processed, (2, 0, 1)).astype(np.float32)  # (C, T, V)
        label = int(sample['label'])
        return torch.from_numpy(processed), torch.tensor(label, dtype=torch.long)


def collate_fn(batch):
    xs, ys = zip(*batch)
    xs = torch.stack(xs, dim=0)
    ys = torch.tensor(ys, dtype=torch.long)
    return xs, ys


# --------------------------
# Model (GraphConv, STGCNBlock, TemporalSelfAttention, STGCN)
# --------------------------
class GraphConv(nn.Module):
    def __init__(self, in_channels, out_channels, num_nodes, bias=True):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.theta = nn.Parameter(torch.Tensor(in_channels, out_channels))
        self.A = nn.Parameter(torch.eye(num_nodes, dtype=torch.float32), requires_grad=True)
        if bias:
            self.bias = nn.Parameter(torch.zeros(out_channels))
        else:
            self.register_parameter('bias', None)
        nn.init.xavier_uniform_(self.theta)

    def forward(self, x):
        B, C, T, V = x.shape
        x_perm = x.permute(0, 2, 3, 1).contiguous().view(B * T, V, C)
        y = x_perm @ self.theta
        A = self.A.unsqueeze(0)
        y = torch.bmm(A.repeat(B * T, 1, 1), y)
        y = y.view(B, T, V, self.out_channels).permute(0, 3, 1, 2).contiguous()
        if self.bias is not None:
            y = y + self.bias.view(1, -1, 1, 1)
        return y


class STGCNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, num_nodes, kernel_size=9, stride=1, residual=True):
        super().__init__()
        self.gcn = GraphConv(in_channels, out_channels, num_nodes)
        padding = (kernel_size - 1) // 2
        self.tcn = nn.Sequential(
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=(kernel_size, 1), padding=(padding, 0), stride=(stride, 1)),
            nn.BatchNorm2d(out_channels),
            nn.Dropout(0.3)
        )
        if not residual:
            self.residual = lambda x: 0
        elif (in_channels == out_channels) and stride == 1:
            self.residual = lambda x: x
        else:
            self.residual = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=(stride,1)),
                nn.BatchNorm2d(out_channels),
            )
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        res = self.residual(x)
        x = self.gcn(x)
        x = self.tcn(x)
        x = x + res
        return self.relu(x)


class TemporalSelfAttention(nn.Module):
    def __init__(self, d_model, n_heads=4):
        super().__init__()
        self.attn = nn.MultiheadAttention(d_model, n_heads, batch_first=True)
        self.ln = nn.LayerNorm(d_model)

    def forward(self, x):
        B, C, T, V = x.shape
        x = x.permute(0, 3, 2, 1).contiguous().view(B * V, T, C)
        out, _ = self.attn(x, x, x)
        out = self.ln(out + x)
        out = out.view(B, V, T, C).permute(0, 3, 2, 1).contiguous()
        return out


class STGCN(nn.Module):
    def __init__(self, in_channels, num_class, num_nodes):
        super().__init__()
        self.data_bn = nn.BatchNorm1d(in_channels * num_nodes)
        self.layers = nn.ModuleList([
            STGCNBlock(in_channels, 64, num_nodes, kernel_size=9, stride=1, residual=False),
            STGCNBlock(64, 64, num_nodes, kernel_size=9, stride=1),
            STGCNBlock(64, 128, num_nodes, kernel_size=9, stride=2),
            STGCNBlock(128, 256, num_nodes, kernel_size=9, stride=2),
        ])
        self.temporal_attn = TemporalSelfAttention(d_model=256, n_heads=4)
        self.pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(256, num_class)

    def forward(self, x):
        B, C, T, V = x.shape
        x = x.permute(0, 1, 3, 2).contiguous()
        x = x.view(B, C * V, T)
        x = self.data_bn(x)
        x = x.view(B, C, V, T).permute(0, 1, 3, 2).contiguous()
        for layer in self.layers:
            x = layer(x)
        x = self.temporal_attn(x)
        x = self.pool(x)
        x = x.view(B, -1)
        out = self.fc(x)
        return out


# --------------------------
# Metrics & Mixup
# --------------------------
def mixup_data(x, y, alpha=1.0, device=None):
    if alpha <= 0:
        return x, y, y, 1.0
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def accuracy_topk(output, target, topk=(1,5)):
    maxk = max(topk)
    batch_size = target.size(0)
    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
        res.append((correct_k.mul_(100.0 / batch_size)).item())
    return res  # list of percentages


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


# --------------------------
# Training pipeline
# --------------------------

def prepare_label_map(split_file, target_num_classes=NUM_CLASSES):
    with open(split_file, 'r') as f:
        data = json.load(f)
    split_dict = data.get('root', data)
    train_actions = set()
    for vid, info in split_dict.items():
        subset = info.get('subset') or info.get('split') or info.get('subset', None)
        if subset == 'train':
            action = info.get('action', [])
            if isinstance(action, list) and len(action) > 0:
                train_actions.add(str(action[0]))
    train_actions = sorted(list(train_actions), key=lambda x: int(x))
    if len(train_actions) != target_num_classes:
        print(f"[Warning] Found {len(train_actions)} unique train actions, expected {target_num_classes}.")
    # build mapping
    label_map = {raw_label: idx for idx, raw_label in enumerate(train_actions)}
    return label_map


In [14]:


def main():
    # build label_map from train split
    label_map = prepare_label_map(SPLIT_FILE, target_num_classes=NUM_CLASSES)

    preprocessor = LandmarkPreprocessor(augment=True, target_frames=T_FRAMES)
    train_dataset = WLASLLandmarkDataset(LANDMARK_FILE, SPLIT_FILE, split='train', preprocessor=preprocessor, label_map=label_map)
    val_dataset = WLASLLandmarkDataset(LANDMARK_FILE, SPLIT_FILE, split='val', preprocessor=preprocessor, label_map=label_map)
    test_dataset = WLASLLandmarkDataset(LANDMARK_FILE, SPLIT_FILE, split='test', preprocessor=preprocessor, label_map=label_map)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True, collate_fn=collate_fn)

    # class weights
    train_labels = [int(s['label']) for s in train_dataset.samples]
    if len(train_labels) == 0:
        raise RuntimeError('No training samples found. Check your split & label_map')
    unique_classes = np.unique(train_labels)
    class_weights = compute_class_weight('balanced', classes=np.arange(len(unique_classes)), y=train_labels)
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(DEVICE)

    # model
    model = STGCN(in_channels=C, num_class=len(unique_classes), num_nodes=TOTAL_LANDMARKS).to(DEVICE)
    print(f"Model has {count_parameters(model)} trainable parameters.")
    optimizer = torch.optim.AdamW(model.parameters(), lr=BASE_LR, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
    scaler = torch.cuda.amp.GradScaler(enabled=USE_AMP)
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor, label_smoothing=LABEL_SMOOTHING)

    best_val = 0.0
    train_history = {'epoch': [], 'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'val_top5': []}

    for epoch in range(EPOCHS):
        model.train()
        running_loss = 0.0
        running_acc = 0.0
        total_samples = 0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} Train")
        for x, y in pbar:
            x = x.to(DEVICE)
            y = y.to(DEVICE)
            batch_size = x.size(0)
            if MIXUP_ALPHA > 0:
                mixed_x, y_a, y_b, lam = mixup_data(x, y, alpha=MIXUP_ALPHA, device=DEVICE)
            else:
                mixed_x, y_a, y_b, lam = x, y, y, 1.0

            optimizer.zero_grad()
            with torch.cuda.amp.autocast(enabled=USE_AMP):
                logits = model(mixed_x)
                loss = lam * criterion(logits, y_a) + (1 - lam) * criterion(logits, y_b)
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()

            # accuracy on hard targets (y)
            acc1 = accuracy_topk(logits.detach().cpu(), y.detach().cpu(), topk=(1,))[0]
            running_loss += loss.item() * batch_size
            running_acc += acc1 * batch_size / 100.0
            total_samples += batch_size
            pbar.set_postfix(loss=running_loss / total_samples, acc=(running_acc / total_samples * 100.0))

        avg_train_loss = running_loss / len(train_dataset)
        avg_train_acc = running_acc / len(train_dataset) * 100.0

        # validation
        model.eval()
        val_loss = 0.0
        val_acc = 0.0
        val_top5 = 0.0
        with torch.no_grad():
            for x, y in tqdm(val_loader, desc="Validation"):
                x = x.to(DEVICE); y = y.to(DEVICE)
                with torch.cuda.amp.autocast(enabled=USE_AMP):
                    logits = model(x)
                    loss = criterion(logits, y)
                val_loss += loss.item() * x.size(0)
                acc1, acc5 = accuracy_topk(logits.detach().cpu(), y.detach().cpu(), topk=(1,5))
                val_acc += acc1 * x.size(0) / 100.0
                val_top5 += acc5 * x.size(0) / 100.0
        avg_val_loss = val_loss / len(val_dataset) if len(val_dataset) > 0 else 0.0
        avg_val_acc = val_acc / len(val_dataset) * 100.0 if len(val_dataset) > 0 else 0.0
        avg_val_top5 = val_top5 / len(val_dataset) * 100.0 if len(val_dataset) > 0 else 0.0

        scheduler.step()

        print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss {avg_train_loss:.4f} Acc {avg_train_acc:.2f}% | Val Loss {avg_val_loss:.4f} Acc {avg_val_acc:.2f}% Top5 {avg_val_top5:.2f}%")

        train_history['epoch'].append(epoch+1)
        train_history['train_loss'].append(avg_train_loss)
        train_history['train_acc'].append(avg_train_acc)
        train_history['val_loss'].append(avg_val_loss)
        train_history['val_acc'].append(avg_val_acc)
        train_history['val_top5'].append(avg_val_top5)

        if avg_val_acc > best_val:
            best_val = avg_val_acc
            torch.save(model.state_dict(), f"best_stgcn_{NUM_CLASSES}.pth")
            print(f"✅ New best model: {best_val:.2f}%")

    print("Training complete. Best val:", best_val)

    # save history
    df = pd.DataFrame(train_history)
    df.to_csv('training_log.csv', index=False)
    print("Training log saved to training_log.csv")

    # curves
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.plot(df['epoch'], df['train_loss'], label='Train Loss')
    plt.plot(df['epoch'], df['val_loss'], label='Val Loss')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend(); plt.title('Loss Curve')
    plt.subplot(1,2,2)
    plt.plot(df['epoch'], df['train_acc'], label='Train Acc')
    plt.plot(df['epoch'], df['val_acc'], label='Val Acc')
    plt.xlabel('Epoch'); plt.ylabel('Accuracy (%)'); plt.legend(); plt.title('Accuracy Curve')
    plt.savefig('training_curves.png')
    plt.close()
    print("Training curves saved to training_curves.png")

    # test
    model.load_state_dict(torch.load(f"best_stgcn_{NUM_CLASSES}.pth", map_location=DEVICE))
    model.eval()
    test_acc = 0.0
    test_top5 = 0.0
    with torch.no_grad():
        for x, y in tqdm(test_loader, desc="Testing"):
            x = x.to(DEVICE); y = y.to(DEVICE)
            with torch.cuda.amp.autocast(enabled=USE_AMP):
                logits = model(x)
            acc1, acc5 = accuracy_topk(logits.detach().cpu(), y.detach().cpu(), topk=(1,5))
            test_acc += acc1 * x.size(0) / 100.0
            test_top5 += acc5 * x.size(0) / 100.0
    avg_test_acc = test_acc / len(test_dataset) * 100.0 if len(test_dataset) > 0 else 0.0
    avg_test_top5 = test_top5 / len(test_dataset) * 100.0 if len(test_dataset) > 0 else 0.0
    print(f"Final Test Accuracy: {avg_test_acc:.2f}% Top5: {avg_test_top5:.2f}%")


if __name__ == '__main__':
    main()


✅ Created 'train' split with 1442 samples.
✅ Created 'val' split with 338 samples.
✅ Created 'test' split with 258 samples.
Model has 1209376 trainable parameters.


Validation: 100%|██████████| 11/11 [00:04<00:00,  2.22it/s]


Epoch 1/200 | Train Loss 4.6159 Acc 0.83% | Val Loss 4.4655 Acc 2.66% Top5 12.13%
✅ New best model: 2.66%


Validation: 100%|██████████| 11/11 [00:04<00:00,  2.28it/s]


Epoch 2/200 | Train Loss 4.3758 Acc 2.43% | Val Loss 4.2024 Acc 5.33% Top5 18.34%
✅ New best model: 5.33%


...

Epoch 199/200 | Train Loss 1.4118 Acc 60.82% | Val Loss 1.7626 Acc 74.26% Top5 91.72%


Validation: 100%|██████████| 11/11 [00:04<00:00,  2.28it/s]


Epoch 200/200 | Train Loss 1.4591 Acc 41.19% | Val Loss 1.7536 Acc 75.44% Top5 91.42%
Training complete. Best val: 76.62721866404517
Training log saved to training_log.csv
Training curves saved to training_curves.png


Testing: 100%|██████████| 9/9 [00:03<00:00,  2.40it/s]

Final Test Accuracy: 72.09% Top5: 88.76%
